# ANÁLISIS DIAGNÓSTICO: Estructura de los CSVs

Primero, vamos a entender exactamente cómo están estructurados los archivos CSV para encontrar el problema.

In [6]:
import pandas as pd
import numpy as np

# Cargar archivos
print("Cargando archivos...")
train = pd.read_csv('./muestra/train_1.csv')
key = pd.read_csv('./muestra/key_1.csv')

print(f"\n=== TRAIN ===")
print(f"Shape: {train.shape}")
print(f"Columnas: {train.columns[:10].tolist()}... ({len(train.columns)} total)")
print(f"\nPrimeras 5 páginas:")
print(train['Page'].head(5).tolist())

print(f"\n=== KEY ===")
print(f"Shape: {key.shape}")
print(f"Columnas: {key.columns.tolist()}")
print(f"\nPrimeras 10 filas:")
print(key.head(10))

Cargando archivos...

=== TRAIN ===
Shape: (145063, 551)
Columnas: ['Page', '2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07', '2015-07-08', '2015-07-09']... (551 total)

Primeras 5 páginas:
['2NE1_zh.wikipedia.org_all-access_spider', '2PM_zh.wikipedia.org_all-access_spider', '3C_zh.wikipedia.org_all-access_spider', '4minute_zh.wikipedia.org_all-access_spider', '52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider']

=== KEY ===
Shape: (8703780, 2)
Columnas: ['Page', 'Id']

Primeras 10 filas:
                                                Page            Id
0  !vote_en.wikipedia.org_all-access_all-agents_2...  bf4edcf969af
1  !vote_en.wikipedia.org_all-access_all-agents_2...  929ed2bf52b9
2  !vote_en.wikipedia.org_all-access_all-agents_2...  ff29d0f51d5c
3  !vote_en.wikipedia.org_all-access_all-agents_2...  e98873359be6
4  !vote_en.wikipedia.org_all-access_all-agents_2...  fa012434263a
5  !vote_en.wikipedia.org_all-access_all-agents_2...  

In [7]:
# Ver una página completa de key (sin truncamiento)
print("=== PÁGINA COMPLETA DE KEY ===")
sample_key_page = key['Page'].iloc[0]
print(f"Length: {len(sample_key_page)}")
print(f"Full: {sample_key_page}")

# Ver 5 páginas de key
print("\n=== PRIMERAS 5 PÁGINAS DE KEY (COMPLETAS) ===")
for i, page in enumerate(key['Page'].head(5)):
    print(f"{i}: {page}")

# Analizar estructura - ver si hay fecha al final
print("\n=== ANÁLISIS DE ESTRUCTURA ===")
sample = key['Page'].iloc[100]
print(f"Sample page: {sample}")
print(f"Length: {len(sample)}")
print(f"Últimos 10 chars: '{sample[-10:]}'")
print(f"Últimos 20 chars: '{sample[-20:]}'")

# Ver algunas páginas de train
print("\n=== PRIMERAS 10 PÁGINAS DE TRAIN ===")
for i, page in enumerate(train['Page'].head(10)):
    print(f"{i}: {page}")

=== PÁGINA COMPLETA DE KEY ===
Length: 55
Full: !vote_en.wikipedia.org_all-access_all-agents_2017-01-01

=== PRIMERAS 5 PÁGINAS DE KEY (COMPLETAS) ===
0: !vote_en.wikipedia.org_all-access_all-agents_2017-01-01
1: !vote_en.wikipedia.org_all-access_all-agents_2017-01-02
2: !vote_en.wikipedia.org_all-access_all-agents_2017-01-03
3: !vote_en.wikipedia.org_all-access_all-agents_2017-01-04
4: !vote_en.wikipedia.org_all-access_all-agents_2017-01-05

=== ANÁLISIS DE ESTRUCTURA ===
Sample page: !vote_en.wikipedia.org_all-access_spider_2017-02-10
Length: 51
Últimos 10 chars: '2017-02-10'
Últimos 20 chars: 'ss_spider_2017-02-10'

=== PRIMERAS 10 PÁGINAS DE TRAIN ===
0: 2NE1_zh.wikipedia.org_all-access_spider
1: 2PM_zh.wikipedia.org_all-access_spider
2: 3C_zh.wikipedia.org_all-access_spider
3: 4minute_zh.wikipedia.org_all-access_spider
4: 52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider
5: 5566_zh.wikipedia.org_all-access_spider
6: 91Days_zh.wikipedia.org_all-access_spider
7: A'N'D_zh.wikipedia

In [9]:
# Estrategia: extraer el nombre base de key (sin la fecha al final)
# y buscar si existe en train

print("=== BÚSQUEDA DE COINCIDENCIAS ===")

# Extraer base de key (quitando la fecha _YYYY-MM-DD)
# Usar apply en lugar de str.rsplit para evitar problemas de sintaxis
key['Page_Base'] = key['Page'].apply(lambda x: x.rsplit('_', 1)[0])

print(f"Bases únicas en key: {key['Page_Base'].nunique()}")
print(f"\nPrimeras 5 bases únicas en key:")
print(key['Page_Base'].unique()[:5])

# Crear conjunto de páginas de train (son las bases directas)
train_pages_set = set(train['Page'].unique())

# Buscar coincidencias
key_bases_set = set(key['Page_Base'].unique())

matches = key_bases_set & train_pages_set
print(f"\n=== RESULTADOS ===")
print(f"Coincidencias encontradas: {len(matches)}")

if matches:
    print(f"\nPrimeras 10 coincidencias:")
    for i, match in enumerate(list(matches)[:10]):
        print(f"  {i+1}: {match}")
    
    # Tomar la primera coincidencia
    nombre_pagina = list(matches)[0]
    print(f"\n✓ Página seleccionada para predicción: {nombre_pagina}")
else:
    print("\n⚠ NO HAY COINCIDENCIAS EXACTAS")
    print("Estrategia alternativa: usar la primera página de train")
    nombre_pagina = train['Page'].iloc[0]
    print(f"✓ Página seleccionada: {nombre_pagina}")
    
print(f"\nEsta página existe en train: {nombre_pagina in train_pages_set}")
print(f"Registros en key para esta página: {(key['Page_Base'] == nombre_pagina).sum()}")

=== BÚSQUEDA DE COINCIDENCIAS ===
Bases únicas en key: 145063

Primeras 5 bases únicas en key:
['!vote_en.wikipedia.org_all-access_all-agents'
 '!vote_en.wikipedia.org_all-access_spider'
 '!vote_en.wikipedia.org_desktop_all-agents'
 '????:Andrey_Belloly_1.jpg_ru.wikipedia.org_all-access_all-agents'
 '????:Andrey_Belloly_1.jpg_ru.wikipedia.org_all-access_spider']

=== RESULTADOS ===
Coincidencias encontradas: 145063

Primeras 10 coincidencias:
  1: Help:Contents/id_www.mediawiki.org_desktop_all-agents
  2: Decathlon_en.wikipedia.org_all-access_spider
  3: 射龍門_(撲克遊戲)_zh.wikipedia.org_all-access_all-agents
  4: 篠原涼子_ja.wikipedia.org_mobile-web_all-agents
  5: Ernest_Sheepshanks_en.wikipedia.org_all-access_all-agents
  6: Julian_Draxler_de.wikipedia.org_all-access_spider
  7: File:WLE_Austria_Logo_(transparent).svg_commons.wikimedia.org_all-access_spider
  8: Hardy_Krüger_de.wikipedia.org_mobile-web_all-agents
  9: Beyoncé_en.wikipedia.org_desktop_all-agents
  10: 駿河太郎_ja.wikipedia.org_des

# Web Traffic Time Series Forecasting

## Objetivos
Este notebook realiza predicciones de tráfico web para una página utilizando un modelo de Regresión Lineal entrenado con datos históricos.

### Pasos principales:
1. **Carga de datos**: Se cargan los archivos CSV de entrenamiento (train) y prueba (key)
2. **Búsqueda de coincidencias**: Se encuentra una página que existe en ambos conjuntos de datos
3. **Preprocesamiento**: Se extrae la serie temporal y se convierten las fechas
4. **Entrenamiento**: Se entrena un modelo de regresión lineal
5. **Predicción**: Se generan predicciones para las fechas futuras
6. **Salida**: Se guarda el resultado en un CSV de submission

## 1. Importar librerías y configurar rutas

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression

# Configuración de rutas
RUTA_DATA = './muestra'          
RUTA_SALIDA = './submissions' 

ARCHIVO_TRAIN = 'train_1.csv'  
ARCHIVO_KEY = 'key_1.csv'
ARCHIVO_SUBMISSION = 'submission_un_caso_final.csv'

# Crear carpeta de salida si no existe
os.makedirs(RUTA_SALIDA, exist_ok=True)

print("--- INICIANDO PROCESO SISTÉMICO (CORRECCIÓN DEFINITIVA DE LÓGICA) ---")

## 2. Cargar datos

In [ ]:
path_train = os.path.join(RUTA_DATA, ARCHIVO_TRAIN)
path_key = os.path.join(RUTA_DATA, ARCHIVO_KEY)

print(f"Cargando archivos CSV desde: {RUTA_DATA}...")

try:
    train_df = pd.read_csv(path_train)
    key_df = pd.read_csv(path_key)
    print(f"✓ Train cargado: {train_df.shape}")
    print(f"✓ Key cargado: {key_df.shape}")
except FileNotFoundError:
    print(f"¡ERROR DE RUTA! Archivos no encontrados.")
    exit()

## 3. Buscar una página en común

Se busca una página que exista en ambos conjuntos de datos (train y key). 
La lógica extrae el nombre base eliminando la fecha con `rsplit('_', 1)[0]`.

In [ ]:
print(">> Buscando una página con datos en ambos sets (Train y Key)...")

# 1. Obtener una lista de páginas base del archivo KEY (sin la fecha)
# CORRECCIÓN DEFINITIVA: rsplit('_', 1)[0] elimina solo el último '_Fecha'
paginas_en_llave = key_df['Page'].apply(lambda x: x.rsplit('_', 1)[0]).unique()

# 2. Buscar la primera página del TRAIN que coincida
nombre_pagina = None
for page_name_train in train_df['Page']:
    page_base_name = page_name_train
    
    # Comprobamos si el nombre base del train existe en la lista de nombres base de la llave
    if page_base_name in paginas_en_llave:
        nombre_pagina = page_name_train
        break

if nombre_pagina is None:
    # Si este error persiste, significa que las 500 páginas de train y key no se cruzan.
    print("¡ERROR FATAL! No se encontró NINGUNA página en común para el análisis de 'un caso único'.")
    print("La lógica de comparación es correcta. Revisa si los sets de datos de muestra son complementarios.")
    exit()

print(f"\n>> Página seleccionada: {nombre_pagina}")

## 4. Extracción y preprocesamiento de la serie temporal

Se extrae la serie histórica de visitas y se convierten las fechas al formato correcto.

In [ ]:
# Extracción y preprocesamiento de la serie temporal
pagina_objetivo = train_df[train_df['Page'] == nombre_pagina].iloc[0]
serie_temporal = pagina_objetivo.drop('Page')
serie_temporal.index = pd.to_datetime(serie_temporal.index)
serie_temporal = pd.to_numeric(serie_temporal, errors='coerce').fillna(0) 

df_model = pd.DataFrame({'visitas': serie_temporal})
df_model['dias'] = (df_model.index - df_model.index[0]).days

print(f"✓ Serie temporal: {len(df_model)} observaciones")
print(f"✓ Rango de fechas: {df_model.index[0].date()} a {df_model.index[-1].date()}")

## 5. Entrenamiento del modelo

Se entrena un modelo de **Regresión Lineal** utilizando los días como característica y las visitas como variable objetivo.

In [ ]:
X_train = df_model[['dias']].values
y_train = df_model['visitas'].values

model = LinearRegression()
model.fit(X_train, y_train)

print(f"✓ Modelo entrenado")
print(f"  - Coeficiente (pendiente): {model.coef_[0]:.6f}")
print(f"  - Intersección: {model.intercept_:.6f}")

## 6. Generación de predicciones

Se realizan predicciones para las fechas futuras en el conjunto de prueba (key).

In [ ]:
print(">> Generando predicciones para el periodo requerido...")

# Filtramos solo las llaves que corresponden a nuestra página objetivo
llaves_caso = key_df[key_df['Page'].str.startswith(nombre_pagina, na=False)].copy()

# Cálculo de días futuros
llaves_caso['Fecha'] = pd.to_datetime(llaves_caso['Page'].apply(lambda x: x[-10:]))
fecha_inicio = df_model.index[0]
dias_futuros = (llaves_caso['Fecha'] - fecha_inicio).days.values.reshape(-1, 1)

# Realizar predicciones
predicciones = model.predict(dias_futuros)
predicciones = np.maximum(predicciones, 0)  # Asegurar valores no negativos

print(f"✓ Predicciones generadas: {len(predicciones)} registros")
print(f"✓ Rango de predicciones: {predicciones.min():.2f} a {predicciones.max():.2f}")

## 7. Guardar resultados

Se crea un DataFrame con los IDs y las predicciones, y se guarda en un archivo CSV.

In [ ]:
submission = pd.DataFrame({
    'Id': llaves_caso['Id'],
    'Visits': predicciones
})

path_guardado = os.path.join(RUTA_SALIDA, ARCHIVO_SUBMISSION)
submission.to_csv(path_guardado, index=False)

print(f"\n>> ¡PROCESO FINALIZADO CON ÉXITO!")
print(f"Archivo de submission para '{nombre_pagina}' guardado en: {path_guardado}")
print("\nMuestra de la salida:")
print(submission.head())

## Análisis de Coincidencias CSV

In [3]:
# Primero contar líneas sin cargar en memoria
train_path = 'muestra/train_1.csv'
key_path = 'muestra/key_1.csv'

print("CONTANDO FILAS EN TRAIN_1.CSV...")
with open(train_path, 'r', encoding='utf-8', errors='ignore') as f:
    train_lines = sum(1 for _ in f) - 1

print(f"Total de filas en train_1.csv: {train_lines:,}")

print("\nCONTANDO FILAS EN KEY_1.CSV...")
with open(key_path, 'r', encoding='utf-8', errors='ignore') as f:
    key_lines = sum(1 for _ in f) - 1

print(f"Total de filas en key_1.csv: {key_lines:,}")

CONTANDO FILAS EN TRAIN_1.CSV...
Total de filas en train_1.csv: 145,063

CONTANDO FILAS EN KEY_1.CSV...
Total de filas en key_1.csv: 8,703,780


In [4]:
import pandas as pd

print("\n" + "="*100)
print("CARGANDO ARCHIVOS PARA ANÁLISIS")
print("="*100)

# Cargar solo primeras filas de train para inspeccionar
print("\nCargando train_1.csv (primeras 500 filas)...")
train_sample = pd.read_csv(train_path, nrows=500)
print(f"✓ Cargado")

print("Cargando key_1.csv completo...")
key_df = pd.read_csv(key_path)
print(f"✓ Cargado ({len(key_df):,} filas)")

print("\n" + "="*100)
print("ESTRUCTURA DE LOS ARCHIVOS")
print("="*100)

print(f"\nTRAIN_1.CSV (muestra de 500 filas):")
print(f"  Columnas: {list(train_sample.columns)}")
print(f"  Forma: {train_sample.shape}")

print(f"\nKEY_1.CSV:")
print(f"  Columnas: {list(key_df.columns)}")
print(f"  Forma: {key_df.shape}")

print("\n" + "-"*100)
print("PRIMERAS FILAS DE TRAIN_1.CSV:")
print("-"*100)
print(train_sample.head(10))

print("\n" + "-"*100)
print("PRIMERAS FILAS DE KEY_1.CSV:")
print("-"*100)
print(key_df.head(10))


CARGANDO ARCHIVOS PARA ANÁLISIS

Cargando train_1.csv (primeras 500 filas)...
✓ Cargado
Cargando key_1.csv completo...
✓ Cargado (8,703,780 filas)

ESTRUCTURA DE LOS ARCHIVOS

TRAIN_1.CSV (muestra de 500 filas):
  Columnas: ['Page', '2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07', '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11', '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15', '2015-07-16', '2015-07-17', '2015-07-18', '2015-07-19', '2015-07-20', '2015-07-21', '2015-07-22', '2015-07-23', '2015-07-24', '2015-07-25', '2015-07-26', '2015-07-27', '2015-07-28', '2015-07-29', '2015-07-30', '2015-07-31', '2015-08-01', '2015-08-02', '2015-08-03', '2015-08-04', '2015-08-05', '2015-08-06', '2015-08-07', '2015-08-08', '2015-08-09', '2015-08-10', '2015-08-11', '2015-08-12', '2015-08-13', '2015-08-14', '2015-08-15', '2015-08-16', '2015-08-17', '2015-08-18', '2015-08-19', '2015-08-20', '2015-08-21', '2015-08-22', '2015-08-23', '2015-08-2

In [5]:
# Analizar columnas identificadoras
print("\n" + "="*100)
print("ANÁLISIS DE LA COLUMNA IDENTIFICADORA")
print("="*100)

# Primera columna es el identificador
train_id_col = train_sample.columns[0]
key_id_col = key_df.columns[0]

print(f"\nCOLUMNA EN TRAIN: '{train_id_col}'")
train_values = set(train_sample[train_id_col].astype(str).unique())
print(f"  Valores únicos en muestra: {len(train_values)}")
print(f"  Primeros 15 valores únicos:")
for v in sorted(list(train_values))[:15]:
    print(f"    {v}")

print(f"\nCOLUMNA EN KEY: '{key_id_col}'")
key_values = set(key_df[key_id_col].astype(str).unique())
print(f"  Valores únicos: {len(key_values):,}")
print(f"  Primeros 15 valores únicos:")
for v in sorted(list(key_values))[:15]:
    print(f"    {v}")


ANÁLISIS DE LA COLUMNA IDENTIFICADORA

COLUMNA EN TRAIN: 'Page'
  Valores únicos en muestra: 500
  Primeros 15 valores únicos:
    100毛_zh.wikipedia.org_all-access_spider
    2016年3月9日日食_zh.wikipedia.org_all-access_spider
    2016年夏季奧林匹克運動會_zh.wikipedia.org_all-access_spider
    2016年美國總統選舉_zh.wikipedia.org_all-access_spider
    2NE1_zh.wikipedia.org_all-access_spider
    2PM_zh.wikipedia.org_all-access_spider
    3C_zh.wikipedia.org_all-access_spider
    4minute_zh.wikipedia.org_all-access_spider
    52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider
    5566_zh.wikipedia.org_all-access_spider
    700歲旅程_zh.wikipedia.org_all-access_spider
    91Days_zh.wikipedia.org_all-access_spider
    A'N'D_zh.wikipedia.org_all-access_spider
    AKB48_zh.wikipedia.org_all-access_spider
    ASCII_zh.wikipedia.org_all-access_spider

COLUMNA EN KEY: 'Page'
  Valores únicos: 8,703,780
  Primeros 15 valores únicos:
    !vote_en.wikipedia.org_all-access_all-agents_2017-01-01
    !vote_en.wikipedia.org_